In [ ]:
import numpy as np
import torch
import torch.nn as nn

from utils import lin_reg, sigmoid, relu, correlation_distance, random_combo, random_partition
from reservoirless_computer import ReservoirlessComputer
from convolutional_RLC import CRLC, KS_to_torch, predict

from data import get_lorenz_data, get_KS_data
from visualization import compare, plot_poincare, plot_images, plot_correlations

In [ ]:
dt = 0.02
padding = 0
sub_length = 3
n = 400

train_data, val_data = get_lorenz_data(tf=250, dt=dt)
t_grid = np.linspace(0, val_data.shape[0] * dt, val_data.shape[0])

training_traj, target = KS_to_torch(train_data, padding, sub_length)

In [ ]:
model_1 = ReservoirlessComputer(dim_system=3, dim_reservoir=400, sigma=0.1)
combo = model_1.r_idx_partition[1]
W_in = model_1.W_in


model_2 = CRLC(n, sub_length, padding, sigma=0.1)
model_2.combo = combo
model_2.conv.weight.data = torch.tensor(W_in, dtype=torch.double).unsqueeze(1)

In [ ]:
R_1 = model_1.train(train_data)
print(R_1.shape)
print(R_1)
#predicted = model_1.predict(val_data.shape[0])
print(lin_reg(R_1, train_data))
#compare(predicted, val_data, t_grid)

In [ ]:
model_2.generate_r_states(training_traj)
#print(model_2.r_state) # passes test
R_2 = model_2.training_states.cpu().numpy().transpose().copy()
print(R_2.shape)
print(R_2)
W_out = lin_reg(R_2, train_data)
print(W_out)
# model_2.train_normal_eq(target)
# model_2.linear.weight.data = torch.tensor(model_1.W_out, dtype=torch.float)
# predicted = predict(model_2, val_data.shape[0])
# compare(predicted, val_data, t_grid)

In [ ]:
print(np.sum(np.abs(R_1 - R_2) > 0.000001))